In [ ]:
# Import Necessary Libraries
import os
import re
import json
from amadeus import Client, ResponseError
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.schema import HumanMessage
import warnings
warnings.filterwarnings("ignore")

# Set up API keys
os.environ["OPENAI_API_KEY"] = "sk***A"
os.environ["AMADEUS_API_KEY"] = "E**h"
os.environ["AMADEUS_API_SECRET"] = "Z**0"

# Initialize Amadeus API client
amadeus = Client(
    client_id=os.getenv("AMADEUS_API_KEY"),
    client_secret=os.getenv("AMADEUS_API_SECRET")
)

# Initialize OpenAI model for retrieving airline names dynamically
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), temperature=0, model_name="gpt-3.5-turbo")

# Helper function to format ISO 8601 duration
def format_duration(iso_duration):
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?', iso_duration)
    hours = match.group(1) if match.group(1) else "0"
    minutes = match.group(2) if match.group(2) else "0"
    return f"{int(hours)} hours {int(minutes)} minutes"

# Function to get full airline names from codes using OpenAI
def get_airline_full_name(airline_code):
    prompt = f"Please provide the full name for the airline only '{airline_code}'."
    response = llm([HumanMessage(content=prompt)])
    return response.content.strip() if response else airline_code  # Return the code if response is empty

# Tool: Flight Search
def fetch_flights(origin, destination, departure_date, return_date=None, max_price=None, airline_name =None):
    try:
        # Set a high default max_price if not provided
        max_price = max_price if max_price else 20000
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": departure_date,
            "adults": 1,
            "maxPrice": max_price
        }

        if return_date:
            params["returnDate"] = return_date  # Include return date for round-trip flights

        # Fetch flights from Amadeus API
        response = amadeus.shopping.flight_offers_search.get(**params)
        flights = response.data

        if flights:
            result = []
            for flight in flights[:5]:  # Limit to top 5 results
                if float(flight['price']['total']) <= max_price:
                    # Outbound flight details
                    segments = flight['itineraries'][0]['segments']
                    airline_code = segments[0]['carrierCode']
                    airline = get_airline_full_name(airline_code)  # Get full airline name
                    # Only add flights that match the specified airline, if provided
                    if airline_name and airline and airline.lower() not in airline_name.lower():
                        continue
                    departure_time = segments[0]['departure']['at']
                    arrival_time = segments[-1]['arrival']['at']
                    flight_duration = format_duration(flight['itineraries'][0]['duration'])

                    # Only include return details if a return date is provided
                    if return_date and len(flight['itineraries']) > 1:
                        return_segments = flight['itineraries'][1]['segments']
                        return_departure_time = return_segments[0]['departure']['at']
                        return_arrival_time = return_segments[-1]['arrival']['at']
                        return_duration = format_duration(flight['itineraries'][1]['duration'])
                        return_info = (
                            f"\nReturn Departure: {return_departure_time}\n"
                            f"Return Arrival: {return_arrival_time}\n"
                            f"Return Duration: {return_duration}\n"
                        )
                    else:
                        return_info = ""

                    # Append both outbound and return information (if available) to results
                    result.append(
                        f"Airline: {airline}\nPrice: ${flight['price']['total']}\n"
                        f"Departure: {departure_time}\nArrival: {arrival_time}\n"
                        f"Duration: {flight_duration}{return_info}"
                        "\n----------------------------------------"
                    )
            return "\n\n".join(result) if result else "No flights found within the budget."
        return "No flights found."
    except ResponseError as error:
        return f"An error occurred: {error.response.result}"

# Define the flight search tool
flight_search_tool = Tool(
    name="Flight Search",
    func=lambda input_str: fetch_flights(
        **{k: v for k, v in json.loads(input_str).items() if k in ["origin", "destination", "departure_date", "return_date"]}
    ),
    description="Find flights based on origin, destination, departure date, and return date."
)

# Initialize memory and agent with the flight search tool
memory = ConversationBufferMemory()
agent = initialize_agent(
    tools=[flight_search_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Helper function to extract flight details from user query
def parse_flight_details(query):
    prompt = (
        f"Extract the following details from this flight request:\n"
        f"Origin location (IATA code if possible), Destination location (IATA code if possible), "
        f"Departure date in MM-DD-YYYY format, and Return date if provided.\n\n"
        f"Request: '{query}'\n\n"
        f"Respond in JSON format with 'origin', 'destination', 'departure_date', and 'return_date'. "
        f"Use IATA codes for origin and destination if known, otherwise, use city names."
    )

    # Create a HumanMessage object from the user query
    message = HumanMessage(content=prompt)

    # Pass the HumanMessage to the LLM and get the response
    response = llm([message])

    # Access the 'content' attribute of the AIMessage response
    response_text = response.content.strip()

    # Continue with the rest of your logic using response_text
    try:
        # Attempt to parse the response as JSON
        details = json.loads(response_text)
    except json.JSONDecodeError:
        print("Error: Unable to parse response as JSON.")
        raise ValueError("Invalid JSON response from model.")

    origin = details.get("origin")
    destination = details.get("destination")
    departure_date = details.get("departure_date")
    return_date = details.get("return_date")

    # Ensure all necessary fields were extracted; otherwise, raise an error
    if not all([origin, destination, departure_date]):
        raise ValueError("Incomplete flight details extracted from the query.")

    return origin, destination, departure_date, return_date

# Main function to interact with the agent dynamically based on user input
def get_flight_recommendations():
    # Prompt the user to enter their request
    user_query = input("Enter prompt: ")

    # Parse the details from the user's query
    try:
        origin, destination, departure_date, return_date = parse_flight_details(user_query)
    except ValueError as e:
        print(f"Error parsing flight details: {e}")
        return

    # Call the agent with structured flight details
    flight_details = {
        "origin": origin,
        "destination": destination,
        "departure_date": departure_date,
        "return_date": return_date
    }
    response = agent.run(json.dumps(flight_details))
    print(response)

# Run the function to allow user input and fetch flight information
get_flight_recommendations()
# Enter prompt: recommend me flights from baltimore to new york on november 23rd, 2024 and return on november 25th, 2024